In [1]:
# import tools from WBT module

import sys
sys.path.insert(1, '/Users/madeleinegallop/Documents/ConnPlanning/wbt_starter')     # path points to my WBT directory

from WBT.whitebox_tools import WhiteboxTools

# declare a name for the tools

wbt = WhiteboxTools()


In [2]:
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Working directories
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Define personal storage root.
# This is the path where you will store inputs and outputs from this workflow.
# For example, my root points to the directory (folder) of s23 in GEOG0310 
# on an external drive named drosera. 

root = "/Users/madeleinegallop/Documents/ConnPlanning"

# Set up separate directories to store temporary and keeper outputs. //writing to harddrive each time

temps = root+"/temps/"     
keeps = root+"/keeps/"   

valleys = keeps+"_0203_valley_bottoms_smoothed_clumped.tif" 
blocks = keeps+"_0353_forested_habitat_blocks_acres.tif" #source was 3m
#for maitenece, these need to be the same 

In [3]:
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Upscale blocks to match 10m extent of valleys 
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

wbt.resample(
    inputs = blocks, 
    output = temps+"/_0401_resample.tif", 
    cell_size=None, #defined with the mama 
    base= valleys, 
    method="nn"
);

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Get blocks without forest 
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
wbt.equal_to(
    input1 = temps+"_0401_resample.tif",
    input2 = 0,
    output = temps+"_0411_inverse_binary.tif")
    
wbt.multiply(
    input1 = temps+"/_0411_inverse_binary.tif",
    input2 = valleys,
    output = temps+"/_0412_valleys_not_blocks.tif",)
    
wbt.clump(
    i = temps+"/_0412_valleys_not_blocks.tif",
    output = temps+"/_0413_valleys_not_blocks_clumps.tif",
    diag = True,
    zero_back = True
)

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Use a kernal to grow the blocks 
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

wbt.maximum_filter(
    i = temps+"_0413_valleys_not_blocks_clumps.tif", 
    output = temps+"_0414_valleys_not_blocks_clumps_extra_edge.tif", 
    filterx=3, 
    filtery=3
)

# Mask background.

wbt.set_nodata_value(
    i = temps+"_0414_valleys_not_blocks_clumps_extra_edge.tif", 
    output = temps+"_0415_valleys_not_blocks_extra_edge_clumps_bg_masked.tif", 
    back_value=0.0,
)



./whitebox_tools --run="Resample" --inputs='/Users/madeleinegallop/Documents/ConnPlanning/keeps/_0353_forested_habitat_blocks_acres.tif' --output='/Users/madeleinegallop/Documents/ConnPlanning/temps//_0401_resample.tif' --base='/Users/madeleinegallop/Documents/ConnPlanning/keeps/_0203_valley_bottoms_smoothed_clumped.tif' --method=nn -v --compress_rasters=False

****************************
* Welcome to Resample      *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress:

0

In [ ]:
#1: find all the areas where there is natural land cover 
#2: isolate the pieces that do not overlap the isolated forest areas. 
#3: add a buffert these isolated areas 
#4: identify ones that are not contiguous at all 
#5: identify ones that are